Following: https://openpolicedata.readthedocs.io/en/stable/getting_started/index.html

In [2]:
# %pip install openpolicedata

In [1]:
import openpolicedata as opd
import pandas as pd

In [12]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
# table_types = opd.datasets.get_table_types(contains="STOPS")
# print(table_types)

['PEDESTRIAN STOPS', 'STOPS', 'STOPS - INCIDENTS', 'STOPS - SUBJECTS', 'TRAFFIC STOPS', 'TRAFFIC STOPS - INCIDENTS', 'TRAFFIC STOPS - SUBJECTS']


In [ ]:
# Load traffic stops data from Lousiville for the year 2022.
src = opd.Source("Cedar Lake")
tbl = src.load(table_type="ARRESTS" ,year=2025)

In [4]:
tbl.table.head()

,Date,Time,Age,Sex,Race,City,State,Statute,Offense,Incident #
0,2025-05-08,7:39,35,F,W,CROWN POINT,IN,35-45-1-3 (90C),DISORDERLY CONDUCT - PUBLIC ORDER,25CL1812
1,2025-05-08,7:30,35,F,W,CROWN POINT,IN,35-45-1-3 (90C),DISORDERLY CONDUCT - PUBLIC ORDER,25CL1812
2,2025-05-05,13:50,53,M,W,PORTAGE,IN,9-24-19-2,DRIVING WHILE SUSPENDED - PRIOR SUSPENSION WIT...,25CL1762
3,2025-05-03,23:47,36,M,W,CHICAGO RIDGE,IL,35-42-2-1 (13A),BATTERY - AGGRAVATED - W/INJURY,25CL1741
4,2025-05-03,23:47,27,M,W,San Tan Valley,AZ,35-42-2-1 (13A),BATTERY - AGGRAVATED - W/INJURY,25CL1741


In [ ]:
# View the 1st 5 rows with pandas' head function
# tbl.table.head()

Finding Datasets
OPD provides the datasets module for querying what datasets are available in OPD. To get all available datasets, query the source table with no inputs:

In [10]:
all_datasets = opd.datasets.query()

# Brainstorming

In [20]:
#get all unique values of datatype
all_datasets['DataType'].unique()

array(['CSV', 'ArcGIS', 'Socrata', 'CKAN', 'Excel', 'HTML', 'Carto'],
      dtype=object)

In [4]:
all_datasets.filter(items=['source_url']).head(10)

,source_url
0,https://data.chandlerpd.com/catalog/arrest-boo...
1,https://data.chandlerpd.com/catalog/calls-for-...
2,https://data.chandlerpd.com/catalog/general-of...
3,https://data.gilbertaz.gov/maps/2dcb4c20c9a444...
4,https://data.gilbertaz.gov/datasets/TOG::gilbe...
5,https://openpolicing.stanford.edu/data/
6,https://data.mesaaz.gov/Police/Police-Computer...
7,https://data.mesaaz.gov/Police/Police-Dispatch...
8,https://data.mesaaz.gov/Police/Police-Incident...
9,https://openpolicing.stanford.edu/data/


Script could have two functions.

## Func 1
1. Args:
    a. Required: URL
    b. Optional: years to try (range), test forward or back(default back), how many years to test for (default 5), specify range where values are found (like substring(url, 3,8))
2. Pull four consecutive digits from URL
    a. Throw error if more than one set of four consec. is found
3. Replace pulled digits forward or backward
4. Test if 200 is returned for CSV/Excel or ArcGIS
    - if No, print status and move to next year to test if applicable
    - if Yes, add to file

## Func 2
1. Filter source spreadsheet by testable DataType
    - CSV, Excel, ArcGIS
2. Filter dataset on URLs that have 1 set of four consective digits: `dataset['URL'].str.contains(r'^\D*(\d{4})\D*$', regex=True)`
3. Check "Last Checked Date" (by surragate key for police_data_source_tracking.csv, `columns_to_match = ['State', 'SourceName', 'AgencyFull', 'TableType']`)
    - Determine "outdated" sources to try depending on dates. i.e. if date is older than a day, month or year
4. If Last Checked is outdated, fetch max coverage end then corresponding URL
5. replace year in url with coverage_end year + 1
6. try to open new url and return data found true or false
    - if No, print status and move to next year to test if applicable
    - if Yes, add to file

# Look for Predictable URLS

##### URLs with only one set of 4 sequential digits?

In [ ]:
# Filter URLs with exactly one set of 4 sequential digits
filtered_records = all_datasets[all_datasets['URL'].str.contains(r'^\D*(\d{4})\D*$', regex=True)]

# Display the filtered dataset
filtered_records[['State', 'SourceName', 'coverage_start', 'coverage_end', 'Year', 'last_coverage_check', 'AgencyFull', 'TableType', 'source_url', 'URL']].tail(20)

C:\Users\Ashley\AppData\Local\Temp\ipykernel_7788\145262043.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  filtered_records = all_datasets[all_datasets['URL'].str.contains(r'^\D*(\d{4})\D*$', regex=True)]


,State,SourceName,coverage_start,coverage_end,Year,last_coverage_check,AgencyFull,TableType,source_url,URL
1428,Vermont,State Police,2017-01-01,2017-12-31,2017,07/07/2023,Vermont State Police,TRAFFIC STOPS,https://vsp.vermont.gov/communityaffairs/trafficstops,https://vsp.vermont.gov/sites/vsp/files/documents/2017trafficstops_WebsiteFinal.xls
1429,Vermont,State Police,2018-01-01,2018-12-31,2018,07/07/2023,Vermont State Police,TRAFFIC STOPS,https://vsp.vermont.gov/communityaffairs/trafficstops,https://vsp.vermont.gov/sites/vsp/files/documents/2018trafficstops_WebsiteFinal.xlsx
1430,Vermont,State Police,2019-01-01,2019-12-31,2019,07/07/2023,Vermont State Police,TRAFFIC STOPS,https://vsp.vermont.gov/communityaffairs/trafficstops,https://vsp.vermont.gov/sites/vsp/files/documents/2019trafficstops_websitefinal.xlsx
1431,Vermont,State Police,2020-01-01,2020-12-31,2020,07/07/2023,Vermont State Police,TRAFFIC STOPS,https://vsp.vermont.gov/communityaffairs/trafficstops,https://vsp.vermont.gov/sites/vsp/files/documents/2020Trafficstops_websitefinal.xlsx
1450,Virginia,Fairfax County,2019-01-01,2019-12-31,2019,07/07/2023,Fairfax County Police Department,ARRESTS,https://www.fcpod.org/pages/crime-data,https://www.fairfaxcounty.gov/police/sites/police/files/Assets/ArcGIS/FCPoD_Site/2019_Arrest_Data.zip
1451,Virginia,Fairfax County,2020-01-01,2020-12-31,2020,07/07/2023,Fairfax County Police Department,ARRESTS,https://www.fcpod.org/pages/crime-data,https://www.fairfaxcounty.gov/police/sites/police/files/Assets/ArcGIS/FCPoD_Site/2020_Arrest_Data.zip
1452,Virginia,Fairfax County,2021-01-01,2021-12-31,2021,07/07/2023,Fairfax County Police Department,ARRESTS,https://www.fcpod.org/pages/crime-data,https://www.fairfaxcounty.gov/police/sites/police/files/Assets/ArcGIS/FCPoD_Site/2021_Arrest_Data.zip
1453,Virginia,Fairfax County,2022-01-01,2022-12-31,2022,07/07/2023,Fairfax County Police Department,ARRESTS,https://www.fcpod.org/pages/crime-data,https://www.fairfaxcounty.gov/police/sites/police/files/Assets/ArcGIS/FCPoD_Site/2022_Arrest_Data.zip
1454,Virginia,Fairfax County,2023-01-01,2023-12-31,2023,05/31/2024,Fairfax County Police Department,ARRESTS,https://www.fcpod.org/pages/crime-data,https://www.fairfaxcounty.gov/police/sites/police/files/Assets/arcgis/fcpod_site/2023_arrest_data.zip
1455,Virginia,Fairfax County,2024-01-01,2024-12-31,2024,03/09/2025,Fairfax County Police Department,ARRESTS,https://www.fcpod.org/pages/crime-data,https://www.fairfaxcounty.gov/police/sites/police/files/assets/arcgis/fcpod_site/2024_arrest_data.zip


##### Edge Case
- URLs are predictable, just need to try changing more than the year in some cases, i.e. "https://data-openjustice.doj.ca.gov/sites/default/files/dataset/2024-12/RIPA-Stop-Data-2023.zip"
- https://www.winooskivt.gov/DocumentCenter/View/780/2018-raw-traffic-stop-and-ticket-data-spread-sheet is downloading when changing to https://www.winooskivt.gov/DocumentCenter/View/780/2019-raw-traffic-stop-and-ticket-data-spread-sheet

### Ideas
- Add unique key to the above table
- Add column for how the source was received, i.e. public record, public records request, etc.
- normalize OPD datasets
    - generate unique keys based on surrogate, last coverage check can apply to key
    - Idk how this would effect package modules
- Are there git checks for PRs on source table? 

# Testing Code

In [ ]:
import datetime
import requests

In [ ]:
# Load the source file containing the dataset URLs to test
df = filtered_records.head(1)

In [ ]:
df

,State,SourceName,Agency,AgencyFull,TableType,coverage_start,coverage_end,last_coverage_check,Year,agency_originated,supplying_entity,Description,source_url,readme,URL,DataType,date_field,dataset_id,agency_field,min_version,py_min_version,query
73,California,California,MULTIPLE,NaN,USE OF FORCE - INCIDENTS,2023-01-01,2023-12-31,09/04/2024,2023,yes,California Office of the Attorney General,The Use of Force Incident Reporting incidents that result in serious bodily injury or death or involved the discharge of a firearm are reported annually from LEAs and other entities throughout the state that employ peace officers. The Use of Force Incident Reporting data is narrowly defined and does not represent the totality of use of force incidents that occur in California.,https://openjustice.doj.ca.gov/data,https://data-openjustice.doj.ca.gov/sites/default/files/dataset/2024-07/use-of-force-readme-06202024f.pdf,https://data-openjustice.doj.ca.gov/sites/default/files/dataset/2024-07/UseofForce_Incident_2023.csv,CSV,INCIDENT_DATE_STR,NaN,ORI,<NA>,<NA>,NaN


### BestieGPT code to start with for functions

TO-DO: review OPD's how-to for contributing to source table and see if funcs can follow that method 

In [ ]:
# Define the threshold for outdated sources
def is_outdated(last_checked, threshold_days=1):
    """Check if the last checked date is older than the threshold."""
    last_checked_date = pd.to_datetime(last_checked)
    return (datetime.datetime.now() - last_checked_date).days > threshold_days

# Iterate through the DataFrame to process outdated sources
for index, row in df.iterrows():
    last_checked = row.get("Last Checked Date")
    if pd.isnull(last_checked) or is_outdated(last_checked):
        print(f"Processing outdated source: {row['URL']}")

        # Fetch max coverage end year
        coverage_end = row.get("coverage_end")
        if pd.isnull(coverage_end):
            print(f"Skipping source {row['URL']} due to missing coverage_end.")
            continue

        # Extract year from the URL
        source_url = row["source_url"]
        year_in_url = None
        for part in source_url.split("/"):
            if part.isdigit() and len(part) == 4:  # Check for a 4-digit year
                year_in_url = int(part)
                break

        if year_in_url is None:
            print(f"No year found in URL: {source_url}")
            continue

        # Replace year in URL with coverage_end year + 1
        new_year = int(coverage_end) + 1
        updated_url = source_url.replace(str(year_in_url), str(new_year))
        print(f"Updated URL: {updated_url}")
        # Try to open the new URL
        try:
            response = requests.get(updated_url)
            if response.status_code == 200:
                print(f"Data found at {updated_url}")
                df.at[index, "Last Checked Date"] = datetime.datetime.now().strftime("%Y-%m-%d")
            
                # Add a new row by copying the current row and updating the related data columns
                new_row = row.copy()
                new_row["coverage_start"] = pd.to_datetime(row["coverage_end"]) + pd.Timedelta(days=1)
                new_row["coverage_end"] = pd.to_datetime(row["coverage_end"]) + pd.DateOffset(years=1)
                new_row["source_url"] = updated_url
                new_row["last_coverage_check"] = datetime.datetime.now().strftime("%m/%d/%Y")
                new_df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
            else:
                print(f"Failed to fetch data from {updated_url}. Status code: {response.status_code}")
        except Exception as e:
            print(f"Error fetching data from {updated_url}: {e}")

Processing outdated source: https://www.beloitwi.gov/index.asp?SEC=3209CC74-34C0-4358-A679-9B061EE965C3&DE=FA507A9C-2ED5-4E8C-B098-8F3E78568AF6
No year found in URL: https://www.beloitwi.gov/index.asp?SEC=3209CC74-34C0-4358-A679-9B061EE965C3&DE=FA507A9C-2ED5-4E8C-B098-8F3E78568AF6
Processing outdated source: https://openpolicing.stanford.edu/data/
No year found in URL: https://openpolicing.stanford.edu/data/
Processing outdated source: https://data.milwaukee.gov/dataset/fpc-citizen-complaints
No year found in URL: https://data.milwaukee.gov/dataset/fpc-citizen-complaints
Processing outdated source: https://openpolicing.stanford.edu/data/
No year found in URL: https://openpolicing.stanford.edu/data/
Processing outdated source: https://openpolicing.stanford.edu/data/
No year found in URL: https://openpolicing.stanford.edu/data/
